<a href="https://colab.research.google.com/github/Agniva2006/Gender-Classification-Deep-Learning-Model/blob/main/Task_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Gender Classification Project

## 1.google drive permision check

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 2.Import zip file from drive

In [ ]:
import zipfile

zip_path = "/content/drive/MyDrive/Comys_Hackathon5.zip"
extract_path = "/content/Comys_Hackathon5"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

## 3.Import the libraries

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms,models,datasets
from torchvision.transforms import ToTensor, Normalize, Resize, Compose, RandomHorizontalFlip, ColorJitter
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

## 4.HyperParaMeters

In [ ]:
BATCH_SIZE = 32
LR = 1e-4
EPOCHS = 5
IMAGE_SIZE = 224

## 5.Data Transforms

In [ ]:
train_transform=transforms.Compose([
    transforms.Resize((IMAGE_SIZE,IMAGE_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2,contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

val_transform=transforms.Compose([
    transforms.Resize((IMAGE_SIZE,IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

##6. Find The Directories Of Train and Validation Datasets

In [ ]:
!find /content/Comys_Hackathon5/ -name "train"
!find /content/Comys_Hackathon5/ -name "val"

## 7.breakdown into train and test datasets

In [ ]:
train_dataset = datasets.ImageFolder("/content/Comys_Hackathon5/Comys_Hackathon5/Task_A/train", transform=train_transform)
val_dataset = datasets.ImageFolder("/content/Comys_Hackathon5/Comys_Hackathon5/Task_A/val", transform=val_transform)

## 8. length of training dataset

In [ ]:
len(train_dataset)

## 9. assigned male as 1 and female as 0

In [ ]:
from collections import Counter

targets = [label for _, label in train_dataset]
class_counts = Counter(targets)

print(class_counts)


## 10. Create Weights For each Sample

In [ ]:
import numpy as np
from torch.utils.data import WeightedRandomSampler

class_sample_counts = [class_counts[0], class_counts[1]]
weights = 1. / np.array(class_sample_counts)

sample_weights = [weights[label] for label in targets]


## 11.initialize the weightRandomSampler

In [ ]:
sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True
)


## 12. load the DataLoader

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=sampler,num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

## 13. use the gpu

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## 14. use of pretrained Model (RESNET18)

In [ ]:
model = models.resnet18(pretrained=True)
#dropout
model.fc = nn.Sequential(
    nn.Dropout(0.4),
    nn.Linear(model.fc.in_features, 2)
)

model.to(device)

## 14. Optimizaor and criterion

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

## 15 .Training the dataset

In [ ]:
train_losses, val_losses, train_accuracies, val_accuracies = [], [], [], []

for epoch in range(EPOCHS):
    # -------------------- Training --------------------
    model.train()
    train_loss, correct, total = 0.0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

    train_acc = 100 * correct / total
    train_losses.append(train_loss / total)
    train_accuracies.append(train_acc)

    # -------------------- Validation --------------------
    model.eval()
    val_loss, correct, total = 0.0, 0, 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_acc = 100 * correct / total
    val_losses.append(val_loss / total)
    val_accuracies.append(val_acc)

    # -------------------- Print Epoch Summary --------------------
    print(f"Epoch {epoch+1}/{EPOCHS}: "
          f"Train Loss={train_losses[-1]:.4f}, Val Loss={val_losses[-1]:.4f}, "
          f"Train Acc={train_acc:.2f}%, Val Acc={val_acc:.2f}%")


## 16.Classification Matrix

In [ ]:
print("Final Validation Metrics (Task A - Gender Classification):")
print(classification_report(all_labels, all_preds, target_names=train_dataset.classes))

## 17. Accuracy Matrices

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(all_labels, all_preds)
print(f" Validation Accuracy: {accuracy * 100:.2f}%")

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(all_labels, all_preds)

In [ ]:
from sklearn.metrics import classification_report

model.eval()
train_preds, train_labels = [], []

with torch.no_grad():
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        preds = torch.argmax(outputs, dim=1).cpu()
        train_preds.extend(preds)
        train_labels.extend(labels)

# Report
print("🔍 Classification Report (Training Set):")
print(classification_report(train_labels, train_preds, target_names=train_dataset.classes))


## code testing

# upload image of mbappe

In [ ]:
from google.colab import files
uploaded = files.upload()


# prepare the image

In [ ]:
from PIL import Image
import torchvision.transforms as transforms
import torch

# Replace with the name of the uploaded image
image_path = list(uploaded.keys())[0]

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # standard ImageNet normalization
                         std=[0.229, 0.224, 0.225])
])

img = Image.open(image_path).convert('RGB')
input_tensor = transform(img).unsqueeze(0).to(device)


# evaluation

In [ ]:
model.eval()
with torch.no_grad():
    output = model(input_tensor)
    _, predicted = torch.max(output, 1)
    class_idx = predicted.item()
    class_names = train_dataset.classes  # ['female', 'male'] from your original dataset

print(f"Predicted Gender: {class_names[class_idx]}")


# showing

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(img)
plt.title(f"Predicted Gender: {class_names[class_idx]}")
plt.axis('off')
plt.show()
